In [1]:
#nans set to 0

# Do I want to make the padding value 0 so i ignore all rows with 0??
# should i use a different optimizer for my compiler?? maybe adam

from google.colab import drive
drive.mount('/content/drive/')

In [2]:
import pandas as pd
import os
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential, layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
ROOT_DATA_DIR = '/content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs'
#ROOT_DATA_DIR = '../raw_data/Out_Feature_CSVs'
DATA_SUB_DIRs = ["Train", "Test"]
CLS_LIST = ["Bad", "Good"]

def get_y(data_path, class_list):
    
    data_dict = {"csv_paths":[], "csv_files":[], "labels":[]}
    for i, clss in enumerate(class_list):
        dir_path = os.path.join(data_path, clss)
        list_csvs = [l for l in os.listdir(dir_path) if l.split(".")[-1]=="csv"]
        print(f"Found {len(list_csvs)} CSVs in {dir_path} Directory")
        data_dict["csv_files"].extend(list_csvs)
        data_dict["labels"].extend([i for k in range(len(list_csvs))])
        data_dict["csv_paths"].extend([dir_path for _ in range(len(list_csvs))])
    total_csvs = len(data_dict["csv_files"])
    total_labels = len(data_dict["labels"])
    print(f"Number of csvs : {total_csvs}")
    print(f"Number of Labels : {total_labels}")
    return data_dict

train_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[0]),
                               class_list=CLS_LIST))


test_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[1]),
                               class_list=CLS_LIST))  

Found 65 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Train/Bad Directory
Found 65 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Train/Good Directory
Number of csvs : 130
Number of Labels : 130
Found 17 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Test/Bad Directory
Found 17 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Test/Good Directory
Number of csvs : 34
Number of Labels : 34


In [4]:
train_data.head()

,csv_paths,csv_files,labels
0,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,IMG_9451_02.csv,0
1,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,IMG_9362_03.csv,0
2,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,IMG_9365_01.csv,0
3,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,63.csv,0
4,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,IMG_9347_02.csv,0


In [5]:
#create y_train and y_test as float64 type
y_train = train_data['labels']/1.0
y_test = test_data['labels']/1.0

In [6]:
def csv_to_list_of_lists(csv_path):
    df = pd.read_csv(csv_path)
    df.fillna(0, inplace=True)
    list_of_lists = df.iloc[:,1:].values.tolist()
    return list_of_lists

def get_x(dataframe):
  list_of_csv_files = [os.path.join(row['csv_paths'], row["csv_files"]) for index, row in dataframe.iterrows()]
  X_list = [csv_to_list_of_lists(f) for f in list_of_csv_files]
  return X_list

In [7]:
#create X_train and X_test
X_train = get_x(train_data)
X_test = get_x(test_data)

In [8]:
#pad X_train so all arrays are of the same shape
X_train_pad = pad_sequences(X_train, dtype='float32', padding='post', value=-1000)
print(X_train_pad.shape)

#pad X_test so all arrays are of the same shape
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post', value=-1000, maxlen=X_train_pad.shape[1])
print(X_test_pad.shape)

(130, 175, 2048)
(34, 175, 2048)


#LSTM_Model5 - Learning Rate and many layers
#New
#1
#[0.6656529307365417, 0.5444444417953491]
#[0.7044618129730225, 0.5]
#2
#[0.8229615688323975, 0.5444444417953491]
#[0.8788918256759644, 0.5]
#3
#
#

#Old
#1
#[0.7743281126022339, 0.6499999761581421]
#[0.7172174453735352, 0.6499999761581421]
#2
#[0.814126193523407, 0.5375000238418579]
#[0.8420926928520203, 0.5]
#3
#[0.5910772085189819, 0.6499999761581421]
#[0.5311039686203003, 0.699999988079071]

model_LSTM = Sequential()

model_LSTM.add(layers.Masking(mask_value=-1000))

model_LSTM.add(layers.LSTM(200, activation='tanh', return_sequences=True))
model_LSTM.add(layers.LSTM(150, activation='tanh'))
model_LSTM.add(layers.Dense(125, activation='relu'))
model_LSTM.add(layers.Dense(100, activation='relu'))
model_LSTM.add(layers.Dense(75, activation='relu'))
model_LSTM.add(layers.Dense(60, activation='relu'))
model_LSTM.add(layers.Dense(40, activation='relu'))
model_LSTM.add(layers.Dense(25, activation='relu'))    
model_LSTM.add(layers.Dense(10, activation='relu'))
model_LSTM.add(layers.Dense(1, activation='sigmoid'))

opt = optimizers.RMSprop(learning_rate=0.003)

model_LSTM.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [9]:
#LSTM_Model_March_8_1 (Binary_accuracy)
#1
#[0.31917595863342285, 0.9769230484962463]
#[2.2517054080963135, 0.8529411554336548]
#2


def build_model(input_shape):

  model_LSTM = Sequential()

  model_LSTM.add(layers.Masking(mask_value=-1000, input_shape=input_shape))
  model_LSTM.add(layers.LSTM(64, activation='tanh', return_sequences=True))
  model_LSTM.add(layers.Dropout(0.2))
  model_LSTM.add(layers.LSTM(32, activation='tanh'))
  model_LSTM.add(layers.Dense(8, activation='relu'))
  model_LSTM.add(layers.Dense(4, activation='relu'))
  model_LSTM.add(layers.Dense(1, activation='sigmoid'))

  model_LSTM.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=[tf.keras.metrics.BinaryAccuracy()])
  return model_LSTM

  print(model_LSTM.summary())

model = build_model(input_shape=(X_train_pad.shape[1], X_train_pad.shape[2]))

#LSTM_Model_March_8 (Binary_accuracy)
#NEW
50/50


#1
#[0.1920861303806305, 0.9666666388511658]
#[2.4868457317352295, 0.6153846383094788]
#2
#[0.18366536498069763, 0.9888888597488403]
#[8.832551956176758, 0.4615384638309479]

#1
#[0.12227339297533035, 0.9777777791023254]
#[1.6843267679214478, 0.6538461446762085]

#2
#[0.7209763526916504, 0.800000011920929]
#[2.0451595783233643, 0.6538461446762085]

#3
#[0.27486875653266907, 0.9555555582046509]
#[3.1554067134857178, 0.5]


def build_model(input_shape):

  model_LSTM = Sequential()

  model_LSTM.add(layers.Masking(mask_value=-1000, input_shape=input_shape))
  model_LSTM.add(layers.LSTM(32, activation='tanh', return_sequences=True))
  model_LSTM.add(layers.LSTM(16, activation='tanh'))
  model_LSTM.add(layers.Dense(8, activation='relu'))
  model_LSTM.add(layers.Dense(4, activation='relu'))
  model_LSTM.add(layers.Dense(1, activation='sigmoid'))

  model_LSTM.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=[tf.keras.metrics.BinaryAccuracy()])
  return model_LSTM

  print(model_LSTM.summary())

model = build_model(input_shape=(X_train_pad.shape[1], X_train_pad.shape[2]))

#Fitting the model to the train set
es = EarlyStopping(patience=15)

model_LSTM.fit(X_train_pad, y_train, 
              epochs=175, 
              batch_size=32, 
              verbose=1, 
              callbacks = [es],
              validation_split=0.2,
              shuffle=True)

In [10]:
#Fitting the model to the train set
es = EarlyStopping(patience=50)

model.fit(X_train_pad, y_train, 
          epochs=1750, 
          batch_size=8, 
          verbose=1, 
          callbacks = [es],
          validation_split=0.2,
          shuffle=True)

Epoch 1/1750
13/13 [==============================] - 12s 502ms/step - loss: 0.6764 - binary_accuracy: 0.5739 - val_loss: 1.0499 - val_binary_accuracy: 0.0000e+00
Epoch 2/1750
13/13 [==============================] - 4s 300ms/step - loss: 0.6421 - binary_accuracy: 0.6731 - val_loss: 1.1162 - val_binary_accuracy: 0.0769
Epoch 3/1750
13/13 [==============================] - 4s 293ms/step - loss: 0.6434 - binary_accuracy: 0.6808 - val_loss: 1.0316 - val_binary_accuracy: 0.0769
Epoch 4/1750
13/13 [==============================] - 4s 290ms/step - loss: 0.6662 - binary_accuracy: 0.6214 - val_loss: 1.0305 - val_binary_accuracy: 0.0769
Epoch 5/1750
13/13 [==============================] - 4s 289ms/step - loss: 0.6196 - binary_accuracy: 0.6900 - val_loss: 0.9954 - val_binary_accuracy: 0.0769
Epoch 6/1750
13/13 [==============================] - 4s 294ms/step - loss: 0.6320 - binary_accuracy: 0.6662 - val_loss: 1.0119 - val_binary_accuracy: 0.0769
Epoch 7/1750
13/13 [===========================

In [15]:
model.save("my_model")
model.save_weights("weights.h5")

INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


In [11]:
#Evaluate the model on the train set
model.evaluate(X_train_pad, y_train)

5/5 [==============================] - 1s 199ms/step - loss: 0.3192 - binary_accuracy: 0.9769


[0.31917595863342285, 0.9769230484962463]

In [12]:
#Evaluate the model on the test set
model.evaluate(X_test_pad, y_test)

2/2 [==============================] - 0s 58ms/step - loss: 2.2517 - binary_accuracy: 0.8529


[2.2517054080963135, 0.8529411554336548]

In [13]:
#predictions of the test set compared to the actuals
model.predict(X_test_pad)

array([[9.5897219e-09],
       [5.3629958e-09],
       [5.7195386e-09],
       [5.0999649e-09],
       [1.0000000e+00],
       [9.9991608e-01],
       [6.1792260e-09],
       [5.3606843e-09],
       [5.7303162e-09],
       [5.3801776e-09],
       [5.2649067e-09],
       [5.7939498e-09],
       [1.0000000e+00],
       [9.9999976e-01],
       [1.0000000e+00],
       [5.6648632e-09],
       [5.3022440e-09],
       [1.0000000e+00],
       [9.9999607e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.9999976e-01],
       [1.0000000e+00],
       [9.9817991e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.9999976e-01],
       [9.9979544e-01],
       [1.0000000e+00],
       [1.0000000e+00]], dtype=float32)

In [14]:
y_test

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
30    1.0
31    1.0
32    1.0
33    1.0
Name: labels, dtype: float64